In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import face_recognition

ModuleNotFoundError: No module named 'cv2'

In [ ]:
def detectCascade(gray, frame):
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.1, 3)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
    return frame

: 

In [ ]:
# Load the cascades
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')


: 

In [ ]:
# Initialize video capture
video_capture = cv2.VideoCapture(0)

if not video_capture.isOpened():
    print("Error: Could not open video capture")
    exit()

frame_count = 0

while True:
    ret, frame = video_capture.read()
    if not ret:
        print("Failed to grab frame")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    canvas = detectCascade(gray, frame)
    
    # Save the frame to a file
    cv2.imwrite(f'frame_{frame_count}.png', canvas)
    frame_count += 1

    # Hentikan loop jika ditekan tombol 'q'
    if frame_count >= 10:
        break

video_capture.release()

: 

In [ ]:
from PIL import Image, ImageDraw
def drawImgLandmarks(frame, face_landmarks_list):
    pil_image = Image.fromarray(frame)
    d = ImageDraw.Draw(pil_image)
    index=0
    while index < len(face_landmarks_list):
        for face_landmarks in face_landmarks_list:
            d.line(face_landmarks['chin'],fill=(255,255,255))
            d.line(face_landmarks['left_eyebrow'],fill=(255,255,255))
            d.line(face_landmarks['right_eyebrow'],fill=(255,255,255))
            d.line(face_landmarks['nose_bridge'],fill=(255,255,255))
            d.line(face_landmarks['nose_tip'],fill=(255,255,255))
            d.line(face_landmarks['left_eye'],fill=(255,255,255))
            d.line(face_landmarks['right_eye'],fill=(255,255,255))
            d.line(face_landmarks['top_lip'],fill=(255,255,255))
            d.line(face_landmarks['bottom_lip'],fill=(255,255,255))
        index +=1
    rgb_image = pil_image.convert('RGB')
    rgb_open_cv_image = np.array(pil_image)
    bgr_open_cv_image = cv2.cvtColor(rgb_open_cv_image, cv2.COLOR_RGB2BGR)
    return bgr_open_cv_image[:, :, ::-1].copy()

: 

In [ ]:
def drawImgLandmarks(image, face_landmarks_list):
    for face_landmarks in face_landmarks_list:
        for facial_feature in face_landmarks.keys():
            cv2.circle(image, face_landmarks[facial_feature], 2, (255, 255, 0), -1)
    return image

webcam_video_stream = cv2.VideoCapture(0)
all_face_locations = []

# Loop through every frame in the video
while True:
    ret, current_frame = webcam_video_stream.read()
    if not ret:
        print("Failed to grab frame")
        break

    face_landmarks_list = face_recognition.face_landmarks(current_frame)
    current_frame = cv2.cvtColor(current_frame, cv2.COLOR_BGR2RGB)  # Convert to RGB for Matplotlib

    # Draw landmarks on the current frame
    current_frame_with_landmarks = drawImgLandmarks(current_frame, face_landmarks_list)

    # Display the current frame with landmarks using Matplotlib
    plt.imshow(current_frame_with_landmarks)
    plt.axis('off')
    plt.show()

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

webcam_video_stream.release()


: 

In [ ]:
from tensorflow.keras.preprocessing import image
from keras.models import model_from_json
webcam_video_stream = cv2.VideoCapture(0)
# face_exp_model = model_from_json(open("dataset/model/facial_expression_model_structure.json","r").read())
face_exp_model.load_weights('keras_model.h5')
emotions_label = ('marah', 'muak', 'takut', 'gembira', 'sedih', 'surprise','netral')
all_face_locations = []
while True:
    ret,current_frame = webcam_video_stream.read()
    current_frame_small = cv2.resize(current_frame,(0,0),fx=0.25,fy=0.25)
    all_face_locations = face_recognition.face_locations(current_frame_small,number_of_times_to_upsample=2,model='hog')
    for index,current_face_location in enumerate(all_face_locations):
#membagi tupel untuk mendapatkan empat nilai posisi wajah
        top_pos,right_pos,bottom_pos,left_pos = current_face_location
#cubah besaran posisi agar sesuai dengan ukuran bingkai video
        top_pos = top_pos*4
        right_pos = right_pos*4
        bottom_pos = bottom_pos*4
        left_pos = left_pos*4
    #simpan gambar face
        current_face_image = current_frame[top_pos:bottom_pos,left_pos:right_pos]
#buat rectangle (opsional)
    cv2.rectangle(current_frame,(left_pos,top_pos),(right_pos,bottom_pos),(0,0,255),2)
#preprocess
    current_face_image = cv2.cvtColor(current_face_image, cv2.COLOR_BGR2GRAY)
    current_face_image = cv2.resize(current_face_image, (48, 48))
    img_pixels = image.img_to_array(current_face_image)
    img_pixels = np.expand_dims(img_pixels, axis = 0)
    img_pixels /= 255
#mempredisksi dengan pretraining model
    exp_predictions = face_exp_model.predict(img_pixels)
# menentukan dengan mencari nilai maksimum dari hasil prediksi
    max_index = np.argmax(exp_predictions[0])
#ambil label emosi
    emotion_label = emotions_label[max_index]
    print('Ditemukan face {} bagian top:{},right:{},bottom:{},left:{}dengan emosi:{}'.format(index+1,top_pos,right_pos,bottom_pos,left_pos,emotion_label))
#display
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(current_frame, emotion_label, (left_pos,bottom_pos), font,0.5, (255,255,255),1)
#showing gambar
    cv2.imshow("Webcam Video",current_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
webcam_video_stream.release()
cv2.destroyAllWindows()


: 

: 